### Imports

In [2]:
import blockcypher
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import typing
import time
import pickle
# All the dates in the blockchain are UTC
UTC=pytz.UTC

## Nota, si no funciona pip: python -m ensurepip

In [10]:
# for the debate of static class with static methods vs library: in a jupyter notebook, to create a static class is more coherent. 
# All the methods under the same class and in the same cell. Easier to translate to an Script if it is needed.
class BlckChainAnalyser():
    @staticmethod
    def calculate_balance_from_txr(trxs: list)-> int:
        """Calculate the balance from a list of txr. The function doesn't check if the transactions are in the same block

        Args:
            trxs (list): List of transactions in the format returned by blockcypher API

        Returns:
            int: Sum of all the values in the list of transactions
        """        
        assert trxs != None
        
        starting_balance = trxs[-1]["ref_balance"]-trxs[-1]["value"]
        acc_balance = 0
        for trx in trxs:
            acc_balance = np.sign(trx["tx_output_n"])*trx["value"]+acc_balance
        return acc_balance
    
    @staticmethod
    def trxs_filter_by_date(trxs: list, init_date: datetime, end_date: datetime)-> list:
        """Filter the list of BTC transactions passed by argument and obtained using blockcypher API to be between init_date end_date

        Args:
            trxs (list): List of transactions in the format returned by blockcypher API
            init_date (datetime): Initial datetime of the range. Needs to be UTC
            end_date (datetime): End datetime of the range. Needs to be UTC

        Returns:
            list: A sublist of the initial list containing transactions above or equal to init_date and strictily bellow to end_date. An empty list if there are no
            transactions in that range of dates
        """
        assert init_date.tzname() == "UTC"
        assert end_date.tzname() == "UTC"
        list_filtered = [ trx for trx in trxs if trx["confirmed"]>=init_date and trx["confirmed"]<end_date]              
        #list_filtered = filter((lambda x: x["confirmed"]>=init_date and x["confirmed"]<end_date),trxs)
        return list_filtered
    
    @staticmethod
    def trxs_filter_by_address(trxs: list, wallet_addres: str)-> list:
        """Filters the list of BTC transactions by address

        Args:
            trxs (list): Lists of transactions in the format returned by blockcypher
            address (str): Wallet address

        Raises:
            err: [description]

        Returns:
            list: List of transactions filtered by wallet
        """
        assert len(trxs) > 0
        list_filtered = [trx for trx in trxs if trx['address'] == wallet_addres]
        return list_filtered
    
    @staticmethod
    def get_details_addresses_low_profile(list_addresses: list, wait: float=0.5)-> list:
        """Make n calls to get_address_details being n the len of list_addresses. It will wait [wait] seconds between each call, to avoid ERROR 429 from the server 

        Args:
            list_addresses (list): List containing all the addresses to be query
            wait (float): Seconds between each call to avoid error from the server
        Returns:
            list: list of dict cotaing: {"address": "<address>", "details": {<details returned from the server>}}
        """        
        details_addresses = []
        for addr in list_addresses:
            try:
                details_addresses_temp = blockcypher.get_address_details(addr)
            except Exception as err:
                print(len(details_addresses))
                raise err
                
            details_addresses.append({"address": addr, "details":details_addresses_temp})
            time.sleep(wait)
        return details_addresses

---
### Reading the data of the 500 biggest BTC addresses


In [4]:
# from here: https://bitinfocharts.com/top-100-richest-bitcoin-addresses.html
big_btc_addr = pd.read_csv("../data/500_biggest_btc_add.csv")
# Spliting the address and the alias
result = big_btc_addr['address'].str.split(n=2,expand=True)
big_btc_addr.drop(columns=["address"], inplace=True, errors="ignore")
big_btc_addr['alias'] = result[2]
big_btc_addr['address'] = result[0]
# Droping all the rows in the DF that are NaN
big_btc_addr.dropna(subset=["address"], inplace=True)
list_of_big_addresses = big_btc_addr.address.to_list()
# We are expecting 500 addresses
assert(len(list_of_big_addresses)==500)

In [5]:
addresses_details = BlckChainAnalyser.get_details_addresses_low_profile(list_of_big_addresses[0:10])

In [8]:
print(len(addresses_details))
print(len(addresses_details[5]["details"]["txrefs"]))

10
200


In [24]:
binance_cold_address = "34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo"
binance_cold_trxs = BlckChainAnalyser.trxs_filter_by_address(addresses_details, binance_cold_address)[0]

##### Convert to pandas a transaction list. Binance cold list of transactions 

In [33]:
binance_cold_trxs['unconfirmed_txrefs'] = 0

In [40]:
binance_cold_trxs['details']['unconfirmed_txrefs'] = 0

In [82]:
binance_cold_df = pd.DataFrame.from_dict(binance_cold_trxs['details'])

In [85]:
binance_cold_df_exploded = binance_cold_df.explode('txrefs')

In [86]:
binance_cold_df_exploded

,address,total_received,total_sent,balance,unconfirmed_balance,final_balance,n_tx,unconfirmed_n_tx,final_n_tx,txrefs,hasMore,tx_url,unconfirmed_txrefs
0,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389805796392,84130083091124,25259722705268,0,25259722705268,720,0,720,tx_hash,True,https://api.blockcypher.com/v1/btc/main/txs/,0
0,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389805796392,84130083091124,25259722705268,0,25259722705268,720,0,720,block_height,True,https://api.blockcypher.com/v1/btc/main/txs/,0
0,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389805796392,84130083091124,25259722705268,0,25259722705268,720,0,720,tx_input_n,True,https://api.blockcypher.com/v1/btc/main/txs/,0
0,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389805796392,84130083091124,25259722705268,0,25259722705268,720,0,720,tx_output_n,True,https://api.blockcypher.com/v1/btc/main/txs/,0
0,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389805796392,84130083091124,25259722705268,0,25259722705268,720,0,720,value,True,https://api.blockcypher.com/v1/btc/main/txs/,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389805796392,84130083091124,25259722705268,0,25259722705268,720,0,720,ref_balance,True,https://api.blockcypher.com/v1/btc/main/txs/,0
199,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389805796392,84130083091124,25259722705268,0,25259722705268,720,0,720,spent,True,https://api.blockcypher.com/v1/btc/main/txs/,0
199,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389805796392,84130083091124,25259722705268,0,25259722705268,720,0,720,confirmations,True,https://api.blockcypher.com/v1/btc/main/txs/,0
199,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,109389805796392,84130083091124,25259722705268,0,25259722705268,720,0,720,confirmed,True,https://api.blockcypher.com/v1/btc/main/txs/,0


In [17]:
binance_cold = blockcypher.get_address_details("34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo")

In [29]:
binance_cold 

{'address': '34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo',
 'total_received': 109389805796392,
 'total_sent': 84130083091124,
 'balance': 25259722705268,
 'unconfirmed_balance': 0,
 'final_balance': 25259722705268,
 'n_tx': 720,
 'unconfirmed_n_tx': 0,
 'final_n_tx': 720,
 'txrefs': [{'tx_hash': '03fa98d46b6f893799f530e43ea1e78d876cc5634228345b910788f911050500',
   'block_height': 723021,
   'tx_input_n': -1,
   'tx_output_n': 0,
   'value': 8859,
   'ref_balance': 25259722705268,
   'spent': False,
   'confirmations': 43,
   'confirmed': datetime.datetime(2022, 2, 12, 23, 52, 21, tzinfo=tzutc()),
   'double_spend': False},
  {'tx_hash': '27495a69a481ceee9de978a1a2ff15215a1712b24d87730a87d0ed48f0545169',
   'block_height': 722466,
   'tx_input_n': -1,
   'tx_output_n': 0,
   'value': 2291,
   'ref_balance': 25259722696409,
   'spent': False,
   'confirmations': 598,
   'confirmed': datetime.datetime(2022, 2, 9, 11, 43, 39, tzinfo=tzutc()),
   'double_spend': False},
  {'tx_hash': '0fb5c2c4f49cf

In [27]:
binance_cold_trxs

{'address': '34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo',
 'details': {'address': '34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo',
  'total_received': 109389805796392,
  'total_sent': 84130083091124,
  'balance': 25259722705268,
  'unconfirmed_balance': 0,
  'final_balance': 25259722705268,
  'n_tx': 720,
  'unconfirmed_n_tx': 0,
  'final_n_tx': 720,
  'txrefs': [{'tx_hash': '03fa98d46b6f893799f530e43ea1e78d876cc5634228345b910788f911050500',
    'block_height': 723021,
    'tx_input_n': -1,
    'tx_output_n': 0,
    'value': 8859,
    'ref_balance': 25259722705268,
    'spent': False,
    'confirmations': 41,
    'confirmed': datetime.datetime(2022, 2, 12, 23, 52, 21, tzinfo=tzutc()),
    'double_spend': False},
   {'tx_hash': '27495a69a481ceee9de978a1a2ff15215a1712b24d87730a87d0ed48f0545169',
    'block_height': 722466,
    'tx_input_n': -1,
    'tx_output_n': 0,
    'value': 2291,
    'ref_balance': 25259722696409,
    'spent': False,
    'confirmations': 596,
    'confirmed': datetime.datetime(2022, 2

In [26]:

list_filtered = BlckChainAnalyser.trxs_filter_by_date(binance_cold_trxs["txrefs"], datetime(2021,12,7, tzinfo=UTC), datetime(2021,12,8, tzinfo=UTC))
BlckChainAnalyser.calculate_balance_from_txr(list_filtered)

KeyError: 'txrefs'

In [233]:
addresses_details

[{'address': '34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo',
  'details': {'address': '34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo',
   'total_received': 109389805646552,
   'total_sent': 82841836914231,
   'balance': 26547968732321,
   'unconfirmed_balance': 0,
   'final_balance': 26547968732321,
   'n_tx': 707,
   'unconfirmed_n_tx': 0,
   'final_n_tx': 707,
   'txrefs': [{'tx_hash': 'eca012b2999324d25e6df1f51791ecfad895ebcd70bd5e046a59a9e666abe5f6',
     'block_height': 716185,
     'tx_input_n': -1,
     'tx_output_n': 0,
     'value': 2095,
     'ref_balance': 26547968732321,
     'spent': False,
     'confirmations': 364,
     'confirmed': datetime.datetime(2021, 12, 28, 23, 58, 39, tzinfo=tzutc()),
     'double_spend': False},
    {'tx_hash': '90366abfad683ef83697efa83a308a81dc040936785176536406df425a473818',
     'block_height': 715858,
     'tx_input_n': -1,
     'tx_output_n': 1,
     'value': 11110,
     'ref_balance': 26547968730226,
     'spent': False,
     'confirmations': 691,
     'confi

## Checking the delta of these addresses in the last 24 hours

In [3]:
blockcypher.get_address_details("1NDyJtNTjmwk5xPNhjgAMu4HDHigtobu1s")

{'address': '1NDyJtNTjmwk5xPNhjgAMu4HDHigtobu1s',
 'total_received': 1558453568084231,
 'total_sent': 1558240994905254,
 'balance': 212573178977,
 'unconfirmed_balance': 0,
 'final_balance': 212573178977,
 'n_tx': 1191567,
 'unconfirmed_n_tx': 0,
 'final_n_tx': 1191567,
 'txrefs': [{'tx_hash': 'e37a2f637b28c062fe1d4ed7a00910a2803ecb32285c29636f02d3dd8b94aa92',
   'block_height': 716145,
   'tx_input_n': -1,
   'tx_output_n': 0,
   'value': 398002,
   'ref_balance': 212573178977,
   'spent': False,
   'confirmations': 13,
   'confirmed': datetime.datetime(2021, 12, 28, 17, 4, 29, tzinfo=tzutc()),
   'double_spend': False},
  {'tx_hash': 'bf136ef8b5e92566d6ed67c19712c4afcb15c56fdf0855748e011af49a571991',
   'block_height': 715923,
   'tx_input_n': -1,
   'tx_output_n': 0,
   'value': 19661,
   'ref_balance': 212572780975,
   'spent': False,
   'confirmations': 235,
   'confirmed': datetime.datetime(2021, 12, 27, 2, 57, 31, tzinfo=tzutc()),
   'double_spend': False},
  {'tx_hash': '77ae43

## UNIT TEST
-----

###### from here: https://stackoverflow.com/questions/40172281/unit-tests-for-functions-in-a-jupyter-notebook

In [187]:
import unittest

class TestBlckChainAnalyser(unittest.TestCase):
    
    def test__balance_calc(self):
        """ Testing a real world case, using the binance_cold wallet and the transaction done at 2021-12-07 06:06:09 
            here: https://bitinfocharts.com/bitcoin/block/712989/34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo?__cf_chl_tk=it9k.tfmmjZpvgQ87dWZTXqRvVqvwNg_lGmqJF.iu4Y-1640963834-0-gaNycGzNCeU
        """        
        binance_cold = blockcypher.get_address_details("34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo")
        list_filtered = BlckChainAnalyser.trxs_filter_by_date(binance_cold["txrefs"], datetime(2021,12,7, tzinfo=UTC), datetime(2021,12,8, tzinfo=UTC))
        self.assertEqual(BlckChainAnalyser.calculate_balance_from_txr(list_filtered),-2264653016196)   

unittest.main(argv=[''], verbosity=2, exit=False)

test__balance_calc (__main__.TestBlckChainAnalyser)
Testing a real world case, using the binance_cold wallet and the transaction done at 2021-12-07 06:06:09 ... ok

----------------------------------------------------------------------
Ran 1 test in 0.531s

OK
